# Data munging with pandas

_Adapted from original materials by [manishamde](https://github.com/manishamde)._

Data munging or data wrangling is loosely the process of manually converting or mapping data from one 
"raw" form into another format that allows for more convenient consumption of the data with the help of 
semi-automated tools.

Data munging is basically the hip term for cleaning up a messy data set.

Data munging involves common operations such as: 
 - Indexing
 - Renaming
 - Handling missing values
 - map(), apply(), applymap()
 - New Columns = f(Existing Columns)
 - Basic stats
 - Merge, join
 - Plots

In [5]:
import pandas as pd
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


We'll try the above operations on a very simple dataframe

In [6]:
def defdf():
    df = pd.DataFrame({'int_col' : [1, 2, 6, 8, -1], 
                   'float_col' : [0.1, 0.2, 0.2, 10.1, None], 
                   'str_col' : ['a', 'b', None, 'c', 'a']})
    return df

df = defdf()
df

,float_col,int_col,str_col
0,0.1,1,a
1,0.2,2,b
2,0.2,6,None
3,10.1,8,c
4,NaN,-1,a


## Indexing

### Selecting a subset of columns
 * Select only the float and string columns of the dataframe

In [5]:
df[['float_col', 'str_col']]

,float_col,str_col
0,0.1,a
1,0.2,b
2,0.2,None
3,10.1,c
4,NaN,a


### Conditional indexing
 * Using boolean indexing, select the rows of the dataframe for which float column is larger than 0.15
 * Select the rows for which float column is larger than 0.1 and integer column is larger than 2. Change 'and' by 'or' 
 * Select the rows for which string column is not 'a'

In [6]:
df[df['float_col'] > 0.15]

,float_col,int_col,str_col
1,0.2,2,b
2,0.2,6,None
3,10.1,8,c


In [10]:
df[df['int_col'] > 2]  

,float_col,int_col,str_col
2,0.2,6,None
3,10.1,8,c


In [12]:
df[(df['float_col'] > 0.15) & (df['int_col'] > 2) ]

,float_col,int_col,str_col
2,0.2,6,None
3,10.1,8,c


In [13]:
df[(df['float_col'] > 0.15) | (df['int_col'] > 2) ]

,float_col,int_col,str_col
1,0.2,2,b
2,0.2,6,None
3,10.1,8,c


In [14]:
df[df['str_col'] != 'a']

,float_col,int_col,str_col
1,0.2,2,b
2,0.2,6,None
3,10.1,8,c


## Renaming

 * Use the rename method to rename all three columns
 * Set inplace=True for the changes to affect the existing dataframe

In [23]:
 df.rename(columns={"float_col": "F", "int_col" : "I", "str_col" : "S"},inplace=True)

In [24]:
df

,F,I,S
0,0.1,1,a
1,0.2,2,b
2,0.2,6,None
3,10.1,8,c
4,NaN,-1,a


## Handling missing values

### Drop missing values

 * Use dropna to drop all rows with missing data (NaN). From now on, perform the rest of the exercises on this modified dataframe.

In [32]:
edf = df.dropna()

In [33]:
edf

,float_col,int_col,str_col
0,0.1,1,a
1,0.2,2,b
3,10.1,8,c


### Fill missing values
 * Use fillna to fill missing data. Fill float column with median of column and string column with a character of your choosing. Use inplace to alter the value in the original dataframe.

In [38]:
df['float_col'] = df['float_col'].fillna(df['float_col'].median())

In [39]:
df

,float_col,int_col,str_col
0,0.1,1,a
1,0.2,2,b
2,0.2,6,None
3,10.1,8,c
4,0.2,-1,a


In [7]:
df['int_col'] = df['int_col'].fillna(df['int_col'].median())

In [8]:
df['str_col'] = df['str_col'].fillna('a')

## Vectorized operations: map, apply
### map
The map operation maps the values of a series iterating over each element
 * Use map to generate a series that equals each element of integer column squared

In [44]:
df['int_col'].map(lambda x: x*x )

0     1
1     4
2    36
3    64
4     1
Name: int_col, dtype: int64

In [13]:
df

,float_col,int_col,str_col
0,0.1,1,a
1,0.2,2,b
2,0.2,6,a
3,10.1,8,c
4,NaN,-1,a


### apply
The apply operation applies a function along any axis of the dataframe. 
 - axis=0: apply function to each column
 - axis=1: apply function to each row
Depending on the return type of the function passed to apply(), the result will either be of lower dimension or the same dimension.


 * Use apply on columns to compute the square root of the float and integer columns
 * Use apply on rows to compute the cumulative sum by rows of the elements of the float and integer columns

In [20]:
from numpy import sqrt
def sqrt1(row):
    return sqrt(row['int_col']) if row['int_col'] > 0 else 0 

def sqrt2(row):
    return sqrt(row['float_col']) if row['float_col'] > 0 else 0 

def sqrtun(row):
    return (sqrt(row['int_col']) if row['int_col'] > 0 else 0, sqrt(row['float_col']) if row['float_col'] > 0 else 0) 

def ssum(col):
    return (sqrt(row['int_col']) if row['int_col'] > 0 else 0, sqrt(row['float_col']) if row['float_col'] > 0 else 0) 


print(df.apply(sqrtun,axis=1))
#print(sqrt(df['float_col']))


#print(df.apply(sqrt2,axis=1))


0             (1.0, 0.316227766017)
1     (1.41421356237, 0.4472135955)
2     (2.44948974278, 0.4472135955)
3    (2.82842712475, 3.17804971641)
4                            (0, 0)
dtype: object


### applymap
The applymap operation applies a function to a dataframe that is intended to operate elementwise

 * Use applymap to transform the dataframe in the following manner: duplicate elements of type string ('z' -> 'zz') and compute the exponential of numerical elements. Hint: define first the function fn that needs to be applied.

## New Columns = f(Existing Columns)

Generating new columns from existing columns in a data frame is an integral part of the data mungling workflow. 

### multiple columns as a function of a single column

 * Use map in combination with zip to construct two new columns being the square and third power of the integer column. [Help](http://stackoverflow.com/questions/12356501/pandas-create-two-new-columns-in-a-dataframe-with-values-calculated-from-a-pre)

### single column as a function of multiple columns

 * Use apply to construct a new column sum of the float and integer columns. [Help](http://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe?lq=1)
 * Use apply to construct a new column composed of the concatenation of the string column and the integer column cast to string

### multiple columns as a function of multiple columns

 * Use apply and a function that returns a Series to construct two new columns being the square root of the float and integer columns. [Help](http://stackoverflow.com/questions/10751127/returning-multiple-values-from-pandas-apply-on-a-dataframe)

## Basic stats

### describe

 * Use describe to gather information on the distribution of the float and integer columns.
 * Use boxplot for a visual representation of the same information

## Merge and Join

Pandas supports database-like joins which makes it easy to link data frames.

 * Perform inner, outer, left and right joins of the dataframe with the second dataframe defined below

In [ ]:
df2 = pd.DataFrame({'str_col_2' : ['a','b'], 'int_col_2' : [1, 2]})
df2

## Plots

Pandas is equiped with straightforward wrappers for quick plotting of data

 * Use plot to visualize the values columns of the dataframe defined below
 * Use hist to visualize the distribution of the data in the form of a histogram

In [ ]:
plot_df = pd.DataFrame(np.random.randn(1000,2),columns=['x','y'])
plot_df['y'] = plot_df['y'].map(lambda x : x + 1)
plot_df.head()